<a href="https://colab.research.google.com/github/arun477/NanoGPT-Experiment-/blob/main/nanoGPT_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib
%matplotlib inline

In [14]:
torch.manual_seed(1337)

In [15]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

--2023-02-09 05:35:08--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.005s  

2023-02-09 05:35:09 (227 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [16]:
chars = sorted(list(set(text)))
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}
encode = lambda s: [stoi[i] for i in s]
decode = lambda e: "".join([itos[i] for i in e])
vocab_size = len(chars)

In [17]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data)* 0.9)
train_data = data[:n] # 90% train data
val_data = data[n:]

In [18]:
dropout = 0.2
block_size = 256 # max context length
embd_dim = 384
n_layer = 6
num_heads = 6
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 3e-4
batch_size = 64
eval_iters = 200
max_iters = 5000
eval_interval = 500

In [19]:
# sigle attention head
class Head(nn.Module):
  def __init__(self, embd_dim, head_size):
    super().__init__()
    self.query = nn.Linear(embd_dim, head_size, bias=False)
    self.key = nn.Linear(embd_dim, head_size, bias=False)
    self.val = nn.Linear(embd_dim, head_size, bias=False)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
  
  def forward(self, x):
    B, T, C = x.shape
    q = self.query(x) # (B, T, head_size)
    k = self.key(x) # (B, T, head_size)
    v = self.val(x) # (B, T, head_size)
    wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    wei = self.dropout(wei) # (B, T, T)
    out = wei @ v # (B, T, head_size)
    return out

In [20]:
# mulit head attention (just a compose of single head attention)
class MultiHeadAttention(nn.Module):
  def __init__(self, embd_dim, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(embd_dim, head_size) for _ in range(num_heads)])
    self.dropout = nn.Dropout(dropout)
    self.proj = nn.Linear(embd_dim, embd_dim)
  
  def forward(self, x):
    out = [h(x) for h in self.heads]
    out = torch.cat(out, dim=-1)
    out = self.proj(out)
    out = self.dropout(out)
    return out

In [21]:
class FeedForward(nn.Module):
  def __init__(self, embd_dim):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(embd_dim, 4*embd_dim),
        nn.ReLU(),
        nn.Linear(4*embd_dim, embd_dim),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)

In [22]:
class Block(nn.Module):
  def __init__(self, embd_dim, num_heads):
    super().__init__()
    head_size = embd_dim // num_heads
    self.sa = MultiHeadAttention(embd_dim, num_heads, head_size)
    self.fw = FeedForward(embd_dim)
    self.ln1 = nn.LayerNorm(embd_dim)
    self.ln2 = nn.LayerNorm(embd_dim)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.fw(self.ln2(x))
    return x

In [23]:
class BigramModel(nn.Module):
  def __init__(self,):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, embd_dim)
    self.position_embedding_table = nn.Embedding(block_size, embd_dim)
    self.blocks = nn.Sequential(*[Block(embd_dim, num_heads) for _ in range(n_layer)])
    self.lf = nn.LayerNorm(embd_dim)
    self.lm_head = nn.Linear(embd_dim, vocab_size)

  def forward(self, idx, target=None):
    B, T = idx.shape

    token_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = token_emb + pos_emb
    x = self.blocks(x)
    x = self.lf(x)
    logits = self.lm_head(x)

    if target is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      target = target.view(B*T)
      loss = F.cross_entropy(logits, target)
    
    return loss, logits

In [24]:
model = BigramModel()
m = model.to(device)

In [25]:
n_params = sum(ele.numel() for ele in m.parameters())/1e6
print(n_params, 'M parameters')

10.788929 M parameters


In [26]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [27]:
def get_batch(split):
  data = train_data if split=='train' else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+1+block_size] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

In [28]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      loss, logits = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [30]:
for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch('train')
  loss, logits = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

step 0: train loss 4.2853, val loss 4.2827
step 500: train loss 2.0071, val loss 2.0987
step 1000: train loss 1.5960, val loss 1.7798
step 1500: train loss 1.4354, val loss 1.6408
step 2000: train loss 1.3449, val loss 1.5751
step 2500: train loss 1.2737, val loss 1.5286
step 3000: train loss 1.2215, val loss 1.4994
step 3500: train loss 1.1806, val loss 1.4788
step 4000: train loss 1.1418, val loss 1.4819
step 4500: train loss 1.1053, val loss 1.4795


In [41]:
def generate(idx, max_new_tokens):
  for _ in range(max_new_tokens):
    idx_cond = idx[:, -block_size:] # crop idx to the block_size tokens
    loss, logits = model(idx_cond)
    logits = logits[:, -1, :]
    probs = F.softmax(logits, dim=-1)
    idx_next = torch.multinomial(probs, num_samples=1)
    idx = torch.cat((idx, idx_next), dim=1)
  return idx

In [43]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(generate(context, max_new_tokens=10000)[0].tolist()))


PETER:
And a rung, seek you to sanctuit on him
What our valriage; there best brief it persortly,
Some blessed were like you hew 'twixt his king.'
The head of her, shall London, you find him you there.

GLOUCESTER:
She hath tongues thy aexions to do everwean those
A carved conceal a hazant to bear me,
And so a that you: hear him it be so,
As Abhoridal may far your fall,
As laster, that forgives you with she spit!

LADY CAPULET:
The prince of allow wings.

ROMEO:
What is it good?

PAULIXENES:
Mine, patient us night; and roused me,
So much passion our bodies are cat asleep,
Where is noble arms by thus.

VIRGILIA:
And say the envity I prate thinks his renown'd
Cry by me to the head; And for unbeing clot
Is no store was entertainted to me.

VOLUMNIA:
Saw you so.

POLIXENES:
Give me this soil; 'tis ta'en and wherein; like the
open her, her if mercy, we longe on you, good nor with some:

VISCALUS:
Ay, you shall go and swears it his far.

ESCALUS:
Yon are asked with further.

First Lord:
I ma

In [44]:
open('generated_output.txt', 'w').write(decode(generate(context, max_new_tokens=10000)[0].tolist()))

10001